# Coding a spam classifier with naive Bayes

### 1. Imports and pre-processing data

We load the data into a Pandas DataFrame, and then preprocess it by adding a string with the (non-repeated) lowercase words in the email.

In [22]:
import numpy as np

In [23]:
import pandas as pd
emails = pd.read_csv('emails.csv')

In [24]:
emails[:10]

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
5,"Subject: great nnews hello , welcome to medzo...",1
6,Subject: here ' s a hot play in motion homela...,1
7,Subject: save your money buy getting this thin...,1
8,Subject: undeliverable : home based business f...,1
9,Subject: save your money buy getting this thin...,1


In [25]:
def process_email(text):
    text = text.lower()
    return list(set(text.split()))

emails['words'] = emails['text'].apply(process_email)

In [26]:
emails[:10]

,text,spam,words
0,Subject: naturally irresistible your corporate...,1,"[really, organization, gaps, specially, :, let..."
1,Subject: the stock trading gunslinger fanny i...,1,"[fanny, duane, edt, plain, attainder, or, clot..."
2,Subject: unbelievable new homes made easy im ...,1,"[hearing, that, 454, new, minute, you, been, l..."
3,Subject: 4 color printing special request add...,1,"[solicitation, azusa, phone, :, graphix, 91706..."
4,"Subject: do not have money , get software cds ...",1,"[d, with, compatibility, do, ?, marriage, trad..."
5,"Subject: great nnews hello , welcome to medzo...",1,"[devitalize, va, groundsel, inexpiable, nice, ..."
6,Subject: here ' s a hot play in motion homela...,1,"[emai, person, does, assumptions, featured, or..."
7,Subject: save your money buy getting this thin...,1,"[with, thing, 10, that, save, ready, you, ?, g..."
8,Subject: undeliverable : home based business f...,1,"[did, recipient, subject, mon, :, 25, q, @, is..."
9,Subject: save your money buy getting this thin...,1,"[with, thing, 10, that, save, ready, lasts, yo..."


In [27]:
num_emails = len(emails)
num_spam = sum(emails['spam'])

print("Number of emails:", num_emails)
print("Number of spam emails:", num_spam)
print()

# Calculating the prior probability that an email is spam
print("Probability of spam:", num_spam/num_emails)

Number of emails: 5728
Number of spam emails: 1368

Probability of spam: 0.2388268156424581


### 2. Training a naive Bayes model

Our plan is to write a dictionary, and in this dictionary record every word, and its pair of occurrences in spam and ham

In [28]:
model = {}

# Training process
for index, email in emails.iterrows():
    for word in email['words']:
        if word not in model:
            model[word] = {'spam': 1, 'ham': 1}
        if word in model:
            if email['spam']:
                model[word]['spam'] += 1
            else:
                model[word]['ham'] += 1

In [29]:
model['lottery']

{'spam': 9, 'ham': 1}

{'spam': 9, 'ham': 1}

In [30]:
model['sale']

{'spam': 39, 'ham': 42}

{'spam': 39, 'ham': 42}

### 3. Using the model to make predictions

In [31]:
def predict_bayes(word):
    word = word.lower()
    num_spam_with_word = model[word]['spam']
    num_ham_with_word = model[word]['ham']
    return 1.0*num_spam_with_word/(num_spam_with_word + num_ham_with_word)

In [32]:
predict_bayes('lottery')

0.9

0.9

In [33]:
predict_bayes('sale')

0.48148148148148145

0.48148148148148145

In [34]:
def predict_naive_bayes(email):
    total = len(emails)
    num_spam = sum(emails['spam'])
    num_ham = total - num_spam
    email = email.lower()
    words = set(email.split())
    spams = [1.0]
    hams = [1.0]
    for word in words:
        if word in model:
            spams.append(model[word]['spam']/num_spam*total)
            hams.append(model[word]['ham']/num_ham*total)
    prod_spams = np.long(np.prod(spams)*num_spam)
    prod_hams = np.long(np.prod(hams)*num_ham)
    return prod_spams/(prod_spams + prod_hams)

In [35]:
predict_naive_bayes('lottery sale')

0.9638144992048691

0.9638144992048691

In [36]:
predict_naive_bayes('Hi mom how are you')

0.12554358867164464

0.12554358867164464

In [37]:
predict_naive_bayes('Hi MOM how aRe yoU afdjsaklfsdhgjasdhfjklsd')

0.12554358867164464

0.12554358867164464

In [38]:
predict_naive_bayes('meet me at the lobby of the hotel at nine am')

6.964603508395964e-05

6.964603508395964e-05

In [39]:
predict_naive_bayes('enter the lottery to win three million dollars')

0.9995234218677428

0.9995234218677428

In [40]:
predict_naive_bayes('buy cheap lottery easy money now')

0.999973472265966

0.999973472265966

In [41]:
predict_naive_bayes('Grokking Machine Learning by Luis Serrano')

0.4197107645488719

0.4197107645488719

In [43]:
predict_naive_bayes('asdfgh')

0.2388268156424581